In [ ]:
from dotenv import load_dotenv
from llama_index.llms.anthropic import Anthropic
from llama_index.core.settings import Settings
from llama_index.core.llms import ChatMessage
from llama_index.llms.anthropic import Anthropic
from llama_index.core.settings import Settings
from llama_index.core.llms import ChatMessage


In [ ]:

load_dotenv()

loader = TextLoader("catbank.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)